In [4]:
np.array([1,2,3,4,7,15,16,17,19,20])

array([ 1,  2,  3,  4,  7, 15, 16, 17, 19, 20])

In [5]:
import cv2
import numpy as np
from PIL import Image
import pandas
import math
from generate_inputs import TrajectoryDataset
from generate_inputs import device
from generate_inputs import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

In [6]:
train_dataset = TrajectoryDataset('./data/', np.array([1,2,3,4,7,15,16,17,19,20]) , 50, 0.5, math.radians(10))
print(len(train_dataset))
train_dataset[0]['image2']

1
2
3
4
7
15
16
17
19
20
79664


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
# Verifying input dimensions

device = torch.device("cuda")
net = Net()
net = net.to(device)
print(net)

input = torch.randn(1, 2, 224, 224).to(device)
out = net(input)
print(out.shape)

x = np.stack([train_dataset[0]['image1'], train_dataset[0]['image2']])
x = np.expand_dims(x, axis=0)
print(x.shape)

Net(
  (conv_layers): Sequential(
    (0): Conv2d(2, 16, kernel_size=(5, 5), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_feat

In [8]:
# Training X- network

import random
import torch.optim as optim
idxs = list(range(len(train_dataset)))
random.shuffle(idxs)
net_x = Net()
net_x = net_x.to(device)
optimizer = optim.Adam(net_x.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(train_dataset) * 0.8)]
test_idxs = idxs[int(len(train_dataset) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][0]).float().to(device)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_x, "net_x_5.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().to(device)
        out = net_x(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][0]).float().to(device)
        #print(out.shape, target.shape)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        optimizer.zero_grad()
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


Test loss: 0.36631198291796746
New lowest test loss, saved!
Train loss: 0.03342584988960675
Test loss: 0.01736367372123152
New lowest test loss, saved!
Train loss: 0.015423840432477085
Test loss: 0.013642552763790403
New lowest test loss, saved!
Train loss: 0.012527026711385291
Test loss: 0.01075067838451326
New lowest test loss, saved!
Train loss: 0.011179554061966053
Test loss: 0.011217079027612289
Train loss: 0.010151157775106938
Test loss: 0.009977896795610149
New lowest test loss, saved!
Train loss: 0.009430312635643158
Test loss: 0.009140492590288186
New lowest test loss, saved!
Train loss: 0.008794492876977237
Test loss: 0.009097502604283584
New lowest test loss, saved!
Train loss: 0.008293987746055848
Test loss: 0.009685200128311743
Train loss: 0.007836481717686042
Test loss: 0.00838356139947907
New lowest test loss, saved!
Train loss: 0.007483578891063471


In [9]:
# Training Y- network

import random
import torch.optim as optim
idxs = list(range(len(train_dataset)))
random.shuffle(idxs)
net_y = Net()
net_y = net_y.to(device)
optimizer = optim.Adam(net_y.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(train_dataset) * 0.8)]
test_idxs = idxs[int(len(train_dataset) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][1]).float().to(device)
        #criterion = nn.MSELoss()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_y, "net_y_5.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_y(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][1]).float().to(device)
        #criterion = nn.MSELoss()
        optimizer.zero_grad()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


Test loss: 0.14075488200823758
New lowest test loss, saved!
Train loss: 0.003021571555673016
Test loss: 0.0017899226720818346
New lowest test loss, saved!
Train loss: 0.0013904040873345624
Test loss: 0.0009817894780056598
New lowest test loss, saved!
Train loss: 0.0010807394361874686
Test loss: 0.0008168966435165492
New lowest test loss, saved!
Train loss: 0.0009085131168910843
Test loss: 0.000771941862212085
New lowest test loss, saved!
Train loss: 0.0007990699762573126
Test loss: 0.0007288230108186897
New lowest test loss, saved!
Train loss: 0.0007343337109020152
Test loss: 0.0007829512071939746
Train loss: 0.0006840729654594903
Test loss: 0.0006687281932940776
New lowest test loss, saved!
Train loss: 0.0006360923459864891
Test loss: 0.0006757024578346462
Train loss: 0.0005953992967868115
Test loss: 0.0006445537225543682
New lowest test loss, saved!
Train loss: 0.0005658177252392887


In [10]:
# Training Angle- network

import random
import torch.optim as optim
idxs = list(range(len(train_dataset)))
random.shuffle(idxs)
net_angle = Net()
net_angle = net_angle.to(device)
optimizer = optim.Adam(net_angle.parameters(), lr=0.0001)
train_idxs = idxs[:int(len(train_dataset) * 0.8)]
test_idxs = idxs[int(len(train_dataset) * 0.8):]
min_loss = 1
for epoch in range(10):
    train_total = 0
    test_total = 0
    for idx in test_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][2]).float().to(device)
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        test_total += loss.item()
    print("Test loss:", test_total / len(test_idxs))
    if min_loss > test_total / len(test_idxs):
        print("New lowest test loss, saved!")
        torch.save(net_angle, "net_angle_5.pyt")
        min_loss = test_total / len(test_idxs)
    
    for idx in train_idxs:
        x = np.stack([train_dataset[idx]['image1'], train_dataset[idx]['image2']])
        x = torch.tensor(np.expand_dims(x, axis=0)).float().cuda()
        out = net_angle(x)
        target = torch.tensor(train_dataset[idx]['rel_pose'][2]).float().to(device)
        optimizer.zero_grad()
        criterion = nn.L1Loss()
        loss = criterion(out, target)
        loss.backward()
        train_total += loss.item()
        optimizer.step()     
    print("Train loss:", train_total / len(train_idxs))


Test loss: 0.09476261270518156
New lowest test loss, saved!
Train loss: 0.009953528699628567
Test loss: 0.005525380642388258
New lowest test loss, saved!
Train loss: 0.004624983897714913
Test loss: 0.004134728669148424
New lowest test loss, saved!
Train loss: 0.0037004139556491367
Test loss: 0.0035637545741653825
New lowest test loss, saved!
Train loss: 0.0032126437207665773
Test loss: 0.003060672995630258
New lowest test loss, saved!
Train loss: 0.0028946787952134757
Test loss: 0.0028743905745250127
New lowest test loss, saved!
Train loss: 0.002628279705770251
Test loss: 0.0026294893421883727
New lowest test loss, saved!
Train loss: 0.002420206539654327
Test loss: 0.002521162889597588
New lowest test loss, saved!
Train loss: 0.0022744897735447528
Test loss: 0.0024754295891904537
New lowest test loss, saved!
Train loss: 0.0021391579403539744
Test loss: 0.002340622510247319
New lowest test loss, saved!
Train loss: 0.002021595889916851
